### Useful links:
NOTE: Java error seems dependant on worker dying. Therefore recommended that we use python v3.10-3.12. Will need to setup venv and run again 
- [local pyspark installation, windows 11 notebook](https://www.youtube.com/watch?v=yslubfE8IQU)
- [Markdown guide](https://www.markdownguide.org/basic-syntax/)

In [2]:
import os, sys, platform, subprocess, shutil, pathlib, json

# --- 0) HARDEN ENV BEFORE SparkSession IS CREATED ---
# Make sure driver & executors use this exact Python
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

# Put Spark temp on a non-synced, short path (OneDrive can cause grief)
local_tmp = r"C:\spark-tmp" if os.name == "nt" else "/tmp/spark-tmp"
pathlib.Path(local_tmp).mkdir(parents=True, exist_ok=True)
os.environ["SPARK_LOCAL_DIRS"] = local_tmp  # used by Spark core
os.environ["TMPDIR"] = local_tmp             # used by Python workers

# --- 1) PRINT VERSIONS & JAVA ---
print("Python:", sys.version)
print("Platform:", platform.platform())
print("Executable:", sys.executable)
print("JAVA_HOME:", os.environ.get("JAVA_HOME"))
try:
    out = subprocess.check_output(["java", "-version"], stderr=subprocess.STDOUT, text=True)
    print(out)
except Exception as e:
    print("java -version failed:", e)

# --- 2) START SPARK IN LOCAL MODE WITH SAFE SETTINGS ---
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("pyspark-eof-diagnose")
    .master("local[*]")  # explicit local
    .config("spark.local.dir", local_tmp)
    .config("spark.python.worker.reuse", "false")   # avoid stale workers
    .config("spark.sql.execution.arrow.pyspark.enabled", "false")  # rule out Arrow crashes
    .config("spark.ui.showConsoleProgress", "true")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")
print("Spark version:", spark.version)


Python: 3.12.11 (main, Aug  8 2025, 17:05:04) [MSC v.1944 64 bit (AMD64)]
Platform: Windows-11-10.0.26200-SP0
Executable: c:\Users\Tony\OneDrive - Ulster University\Teaching\COM739\2025-26\.venv\Scripts\python.exe
JAVA_HOME: C:\Program Files\Eclipse Adoptium\jdk-17.0.16.8-hotspot\
openjdk version "17.0.16" 2025-07-15
OpenJDK Runtime Environment Temurin-17.0.16+8 (build 17.0.16+8)
OpenJDK 64-Bit Server VM Temurin-17.0.16+8 (build 17.0.16+8, mixed mode, sharing)

Spark version: 4.0.1


In [6]:
spark.stop()

In [3]:
data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]

columns = ["firstname","middlename","lastname","dob","gender","salary"]
df = spark.createDataFrame(data=data, schema = columns)

In [ ]:
# --- 3) SMOKE TEST (should ALWAYS work) ---
print("Basic range collect:")
print(spark.range(5).collect())

# --- 4) YOUR DF: try ultra-small read path ---
print("df.limit(1).collect() test:")
print(df.limit(1).collect())

# --- 5) Narrow: single column, schema, plan ---
print("Schema:")
df.printSchema()
print("\nFirst plan:")
df.explain(True)

first_col = df.columns[0] if df.columns else None
if first_col:
    print("\nSingle column limit(5):")
    print(df.select(first_col).limit(5).collect())

# --- 6) If you use UDFs, try stripping them: select only native cols ---
# Heuristic: find obviously safe primitive columns
safe_types = {"IntegerType","LongType","DoubleType","FloatType","StringType","BooleanType","ShortType","ByteType","DateType","TimestampType"}
from pyspark.sql.types import StructType
schema = df.schema
safe_cols = [f.name for f in schema.fields if f.dataType.simpleString().split("(")[0] in {t.lower().replace("type","") for t in safe_types}]
if safe_cols:
    print("\nSafe cols sample collect:")
    print(df.select(*safe_cols[:min(5,len(safe_cols))]).limit(10).collect())

# --- 7) RDD path test (bypasses some SQL codegen paths) ---
print("\nRDD take(1) test:")
print(df.rdd.take(1))

In [5]:
from pyspark.sql import functions as F, types as T

def simple_dtype(t):
    # normalize type name for easy checks
    return t.simpleString().lower()

schema = df.schema

problem_cols = []
ok_cols = []
for f in schema.fields:
    col = f.name
    try:
        df.select(col).limit(5).collect()
        ok_cols.append((col, simple_dtype(f.dataType)))
    except Exception as e:
        problem_cols.append((col, simple_dtype(f.dataType), repr(e)))

print("OK columns:", ok_cols[:10], "... total:", len(ok_cols))
print("Problem columns:", problem_cols)

# Heuristic: try safe string-casts for problematic ones to confirm it's a serializer/type issue
cast_exprs = []
for f in schema.fields:
    col = f.name
    if any(col == p[0] for p in problem_cols):
        cast_exprs.append(F.col(col).cast("string").alias(f"{col}__as_str"))
    else:
        cast_exprs.append(F.col(col))

print("Try collecting with problematic columns cast to string:")
print(df.select(*cast_exprs).limit(5).collect())


OK columns: [] ... total: 0
Problem columns: [('firstname', 'string', "Py4JJavaError('An error occurred while calling o73.collectToPython.\\n', JavaObject id=o74)"), ('middlename', 'string', "Py4JJavaError('An error occurred while calling o81.collectToPython.\\n', JavaObject id=o82)"), ('lastname', 'string', "Py4JJavaError('An error occurred while calling o89.collectToPython.\\n', JavaObject id=o90)"), ('dob', 'string', "Py4JJavaError('An error occurred while calling o97.collectToPython.\\n', JavaObject id=o98)"), ('gender', 'string', "Py4JJavaError('An error occurred while calling o105.collectToPython.\\n', JavaObject id=o106)"), ('salary', 'bigint', "Py4JJavaError('An error occurred while calling o113.collectToPython.\\n', JavaObject id=o114)")]
Try collecting with problematic columns cast to string:


Py4JJavaError: An error occurred while calling o158.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 55) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker exited unexpectedly (crashed). Consider setting 'spark.sql.execution.pyspark.udf.faulthandler.enabled' or'spark.python.worker.faulthandler.enabled' configuration to 'true' for the better Python traceback.
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:621)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator$$anonfun$1.applyOrElse(PythonRunner.scala:599)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:35)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:945)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:925)
	at

In [1]:
import sys, os
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

In [2]:
""" Configuration of environment for PySpark applications. """
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkTest").getOrCreate()

print(f"findspark version: {findspark.__version__}")
print(f"pyspark version: {spark.version}")

findspark version: 2.0.1
pyspark version: 3.5.7


In [4]:
# read in json file
df1 = spark.read.format("json").load("people.json") # this works although more verbose
df2 = spark.read.json("people.json") # this is a direct way to read json
df2.show()
df1.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [5]:
df1.printSchema() # print the data schema 
df2.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [6]:
df1.collect()
df2.collect() # collect the data as a list of rows

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin')]

In [7]:
df1.count()

3

In [8]:
df1.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   NULL|
| stddev|7.7781745930520225|   NULL|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



## Week 3 pyspark

In [9]:
data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]

columns = ["firstname","middlename","lastname","dob","gender","salary"]
df = spark.createDataFrame(data=data, schema = columns)

In [10]:
df.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)



In [11]:
df.collect()

[Row(firstname='James', middlename='', lastname='Smith', dob='1991-04-01', gender='M', salary=3000),
 Row(firstname='Michael', middlename='Rose', lastname='', dob='2000-05-19', gender='M', salary=4000),
 Row(firstname='Robert', middlename='', lastname='Williams', dob='1978-09-05', gender='M', salary=4000),
 Row(firstname='Maria', middlename='Anne', lastname='Jones', dob='1967-12-01', gender='F', salary=4000),
 Row(firstname='Jen', middlename='Mary', lastname='Brown', dob='1980-02-17', gender='F', salary=-1)]